In [303]:
# Import packages
import os
import pandas as pd

# Set base path for reproducibility
base_path = os.path.join(os.getcwd(), "Capstone Project Data")

# Set file paths for each semester
file_fall21 = os.path.join(base_path, "College of InfoSci_Fall 2021 - CLEANED.xlsx")
file_fall22 = os.path.join(base_path, "College of InfoSci_Fall 2022 - CLEANED.xlsx")
file_fall23 = os.path.join(base_path, "College of InfoSci_Fall 2023 - CLEANED.xlsx")
file_fall24 = os.path.join(base_path, "College of InfoSci_Fall 2024 - CLEANED.xlsx")
file_spring22 = os.path.join(base_path, "College of InfoSci_Spring 2022 - CLEANED.xlsx")
file_spring23 = os.path.join(base_path, "College of InfoSci_Spring 2023 - CLEANED.xlsx")
file_spring24 = os.path.join(base_path, "College of InfoSci_Spring 2024 - CLEANED.xlsx")
file_spring25 = os.path.join(base_path, "College of InfoSci_Spring 2025 - CLEANED.xlsx")


In [304]:
# Load each file into a separate DataFrame

# Fall DFs
df_fall21 = pd.read_excel(file_fall21)
df_fall22 = pd.read_excel(file_fall22)
df_fall23 = pd.read_excel(file_fall23)
df_fall24 = pd.read_excel(file_fall24)

# Spring DFs
df_spring22 = pd.read_excel(file_spring22)
df_spring23 = pd.read_excel(file_spring23)
df_spring24 = pd.read_excel(file_spring24)
df_spring25 = pd.read_excel(file_spring25)

Create Dictionary (dfs) for all our DataFrames. Will use the Dictionary for data cleansing with for-loops

In [305]:
# Dictionary of all our DataFrames
dfs = {
    "df_fall21": df_fall21,
    "df_fall22": df_fall22,
    "df_fall23": df_fall23,
    "df_fall24": df_fall24,
    "df_spring22": df_spring22,
    "df_spring23": df_spring23,
    "df_spring24": df_spring24,
    "df_spring25": df_spring25
}

In [306]:
# Check data types from first DF
print(df_fall21.dtypes)

Term                            object
Session                         object
Session Code                    object
Campus                          object
Facility                        object
Room Capacity                    int64
Subject Code                    object
Catalog Number                  object
Class Section                   object
Req Desig                       object
Component                       object
Min Units                        int64
Max Units                        int64
Combined Section                object
Enrollment Status               object
Instruction Mode                object
Class Number                     int64
Course Description              object
Class Status Code               object
Start Date              datetime64[ns]
End Date                datetime64[ns]
Class Meeting Number           float64
Meeting Days                    object
Meeting Time Start              object
Meeting Time End                object
Total Enroll             

In [307]:
# Check data types for each DF

for df_semester, df in dfs.items():
    print(f"\nData types for {df_semester}:")
    print(df.dtypes)


Data types for df_fall21:
Term                            object
Session                         object
Session Code                    object
Campus                          object
Facility                        object
Room Capacity                    int64
Subject Code                    object
Catalog Number                  object
Class Section                   object
Req Desig                       object
Component                       object
Min Units                        int64
Max Units                        int64
Combined Section                object
Enrollment Status               object
Instruction Mode                object
Class Number                     int64
Course Description              object
Class Status Code               object
Start Date              datetime64[ns]
End Date                datetime64[ns]
Class Meeting Number           float64
Meeting Days                    object
Meeting Time Start              object
Meeting Time End                objec

Each dataframe has the same data types

Update Data Types into Integer: Room Capacity, Min Units, Max Units, Class Number, Class Meeting Number

In [308]:
# Columns to convert to Int64
columns_to_change = ["Room Capacity", "Min Units", "Max Units", "Class Number", "Class Meeting Number"]

# Iterate through our dictionary (dfs), defined above
for df_semester, df in dfs.items():
    df[columns_to_change] = df[columns_to_change].astype("Int64")    # Convert to Int64
    dfs[df_semester] = df                                            # Update dictionary
    print(f"{df_semester}: Selected columns converted to Int64")

# Double check if data updated
#print(df_fall21.dtypes)

df_fall21: Selected columns converted to Int64
df_fall22: Selected columns converted to Int64
df_fall23: Selected columns converted to Int64
df_fall24: Selected columns converted to Int64
df_spring22: Selected columns converted to Int64
df_spring23: Selected columns converted to Int64
df_spring24: Selected columns converted to Int64
df_spring25: Selected columns converted to Int64


Remove unnecessary data:
1) Component > Independent Study
2) Class Status Code > T
3) Req Desig > HONR

In [309]:
for df_semester, df in dfs.items():
    dfs[df_semester] = df[(df["Component"] != "Independent Study") &
                   (df["Class Status Code"] != "T") &
                   (df["Req Desig"] != "HONR")]
    print(f"{df_semester}: Independent Study, T status, and Honor observations removed")


df_fall21: Independent Study, T status, and Honor observations removed
df_fall22: Independent Study, T status, and Honor observations removed
df_fall23: Independent Study, T status, and Honor observations removed
df_fall24: Independent Study, T status, and Honor observations removed
df_spring22: Independent Study, T status, and Honor observations removed
df_spring23: Independent Study, T status, and Honor observations removed
df_spring24: Independent Study, T status, and Honor observations removed
df_spring25: Independent Study, T status, and Honor observations removed


Check for Missing Data in each DataFrame

In [310]:
# Check for missing values 

# Empty dictionary to store the results
missing_data_check = {}

for df_semester, df in dfs.items():
    missing_total = df.isnull().sum().sum() # Total missing values; 1st sum() sums columns, 2nd sums rows
    if missing_total > 0:                   # If missing values found, add to our empty dictionary (missing_data_check)
        missing_data_check[df_semester] = missing_total  # missing_total by df_semester adding to our missing_data_check dictionary

# Print results
if missing_data_check:
    print("DataFrames with missing values:")
    # Iterate through our missing values dictionary (missing_data_check)
    for df_semester, missing_sum in missing_data_check.items():
        print(f"{df_semester}: {missing_sum} missing values")
else:
    print("No missing data found!")

DataFrames with missing values:
df_fall21: 6 missing values
df_fall22: 6 missing values
df_spring22: 6 missing values


Fall21, Fall22, and Spring22 all have missing data

Explore what is missing in them:

In [311]:
#Explore which data is missing in Fall21, Fall22, Spring22; make new dictionary for missing data
dfs_missing = {
    "df_fall21": df_fall21,
    "df_fall22": df_fall22,
    "df_spring22": df_spring22}

for df_semester, df in dfs_missing.items():
    if df.isnull().sum().sum() > 0:                 # If there total sum of missing values found in the entire df > 0
        print(f"\nMissing data in {df_semester}:")
        print(df.isnull().sum())                    # Print each column & whether there is missing values by column


Missing data in df_fall21:
Term                    0
Session                 0
Session Code            0
Campus                  0
Facility                0
Room Capacity           0
Subject Code            0
Catalog Number          0
Class Section           0
Req Desig               0
Component               0
Min Units               0
Max Units               0
Combined Section        0
Enrollment Status       0
Instruction Mode        0
Class Number            0
Course Description      0
Class Status Code       0
Start Date              1
End Date                1
Class Meeting Number    1
Meeting Days            1
Meeting Time Start      1
Meeting Time End        1
Total Enroll            0
Enrollment Capacity     0
dtype: int64

Missing data in df_fall22:
Term                    0
Session                 0
Session Code            0
Campus                  0
Facility                0
Room Capacity           0
Subject Code            0
Catalog Number          0
Class Section        

The Missing Data is the same 6 columns for each of the 3 dataframes (Fall21, Fall22, Spring22):

Start Date              1

End Date                1

Class Meeting Number    1

Meeting Days            1

Meeting Time Start      1

Meeting Time End        1

Replace the Missing Data with Default Values

In [312]:
# Replace the missing data with default values

# Define what we will insert, by DF
default_values_fall21 = {
    "Start Date": pd.Timestamp("2021-08-23 00:00:00"),  #match datetime64[ns] format
    "End Date": pd.Timestamp("2021-12-08 00:00:00"),
    "Class Meeting Number": 1,
    "Meeting Days": "-",
    "Meeting Time Start": "00:00:00",
    "Meeting Time End": "00:00:00"}

default_values_fall22 = {
    "Start Date": pd.Timestamp("2022-08-22 00:00:00"),
    "End Date": pd.Timestamp("2022-12-07 00:00:00"),
    "Class Meeting Number": 1,
    "Meeting Days": "-",
    "Meeting Time Start": "00:00:00",
    "Meeting Time End": "00:00:00"}

default_values_spring22 = {
    "Start Date": pd.Timestamp("2022-01-12 00:00:00"),
    "End Date": pd.Timestamp("2022-05-04 00:00:00"),
    "Class Meeting Number": 1,
    "Meeting Days": "-",
    "Meeting Time Start": "00:00:00",
    "Meeting Time End": "00:00:00"}

#If the cell is "missing" and the Session is "Regular Academic Session", insert the default values defined above
for col, default in default_values_fall21.items():
    df_fall21.loc[(df["Session"] == "Regular Academic Session") & (df_fall21[col].isnull()), col] = default

for col, default in default_values_fall22.items():
    df_fall22.loc[(df["Session"] == "Regular Academic Session") & (df_fall22[col].isnull()), col] = default

for col, default in default_values_spring22.items():
    df_spring22.loc[(df_spring22["Session"] == "Regular Academic Session") & (df_spring22[col].isnull()), col] = default

In [313]:
# Confirm if the missing data was fixed
# Re-run the same For Loop (with dfs_missing) that originally found the missing data
for df_semester, df in dfs2.items():
    if df.isnull().sum().sum() > 0:
        print(f"\nMissing data in {df_semester}:")
        print(df.isnull().sum())
else: 
    print("Missing values fixed!")


Missing values fixed!


Missing values fixed

Next, look for In-Person Classes without Dates or Start/End Times

In [314]:
#Check In-Person Classes without Dates or Start/End Times

days_times_columns = ["Meeting Days", "Meeting Time Start", "Meeting Time End"]

# List to store missing data results & which columns to show in the final summary
filtered_rows = []
filtered_columns_to_show = ["Component", "Class Number", "Course Description", "Meeting Days", "Meeting Time Start", "Meeting Time End"]

# Iterate through each DataFrame in our dictionary "dfs"
for df_semester, df in dfs.items():
    condition = (                                   # Conditions: In Person & not Independent Study           
        (df["Instruction Mode"] == "In Person") &  
        (df["Component"] != "Independent Study")) 
    
    # Find missing observations ("-") in our days_times_columns columns
    missing_days_times = df[condition & df[days_times_columns].isin(["-"]).any(axis=1)] # .any() for any column with "-", not just 3 for 3
    
    # If missing data exists ("-"), store in missing_data_report
    if not missing_days_times.empty:
        missing_data_report[df_semester] = missing_days_times.copy()

# Print results
if missing_data_report:
    for df_semester, df_missing in missing_data_report.items():
        if isinstance(df_missing, pd.DataFrame):    # Confirm df_missing is a DataFrame before data manipulation
            #Set up our empty list (filtered_rows) to store the rows with missing data in our selected columns
            selected_data = df_missing[filtered_columns_to_show].copy() # Create new DF with our selected columns
            selected_data.insert(0, "DataFrame Name", df_semester)      # Insert new column for the Dataframe Name
            filtered_rows.append(selected_data)                         # Add to list filtered_rows by appending selected_data

    # Print results
    if filtered_rows:
        missing_data_summary = pd.concat(filtered_rows, ignore_index = True) # Transform our list (filtered_rows) to DF
    
        print("\nIn-Person Classes without Dates or Start/End Times:")
        print(missing_data_summary)
        

else:
    print("No missing schedule info found!")



In-Person Classes without Dates or Start/End Times:
  DataFrame Name   Component  Class Number           Course Description  \
0      df_fall22  Laboratory         67885    Designing an Installation   
1    df_spring22  Laboratory         95553    Designing an Installation   
2    df_spring25     Lecture         71113  Foundations of Data Science   
3    df_spring25     Lecture         69124    Intro to Machine Learning   

  Meeting Days Meeting Time Start Meeting Time End  
0            -           00:00:00         00:00:00  
1            -           00:00:00         00:00:00  
2            -           00:00:00         00:00:00  
3            -           00:00:00         00:00:00  


5 Classes without Dates or Start/End Times

Decided to Delete them

In [315]:
#Delete the 5 Classes without Dates or Start/End Times

for df_semester, df in dfs.items():
    # Find those 4 Classes again
    condition = (
        (df["Instruction Mode"] == "In Person") &
        (df["Component"] != "Independent Study") &
        (df[days_times_columns].isin(["-"]).any(axis=1))) # Searching for blanks here now
    
    # Drop the rows that meet the condition
    dropped_count = df[condition].shape[0]                # Count dropped rows before deleting
    df.drop(df[condition].index, inplace=True)          # Delete the rows

    # Print how many rows were deleted
    if num_dropped > 0:
        print(f"{dropped_count} rows removed from {df_semester}.")
    else:
        print(f"No rows removed from {df_semester}.")


0 rows removed from df_fall21.
1 rows removed from df_fall22.
0 rows removed from df_fall23.
0 rows removed from df_fall24.
1 rows removed from df_spring22.
0 rows removed from df_spring23.
0 rows removed from df_spring24.
0 rows removed from df_spring25.


/var/folders/qx/l6hqqfxn2yl_2wbqb4x03sf80000gn/T/ipykernel_71042/3282125677.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[condition].index, inplace=True)          # Delete the rows
/var/folders/qx/l6hqqfxn2yl_2wbqb4x03sf80000gn/T/ipykernel_71042/3282125677.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[condition].index, inplace=True)          # Delete the rows
/var/folders/qx/l6hqqfxn2yl_2wbqb4x03sf80000gn/T/ipykernel_71042/3282125677.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

In [316]:
"""
for df_semester, df in dfs.items():
    condition = (                                   # Conditions: In Person & not Independent Study           
        (df["Instruction Mode"] == "In Person") &  
        (df["Component"] != "Independent Study")) 
    
    # Find missing observations ("-") in our days_times_columns columns
    missing_days_times = df[condition & df[days_times_columns].isin(["-"]).any(axis=1)] # .any() for any column with "-", not just 3 for 3
    
    # If missing data exists ("-"), store in missing_data_report
    if not missing_days_times.empty:
        missing_data_report[df_semester] = missing_days_times.copy()

# Print results
if missing_data_report:
    for df_semester, df_missing in missing_data_report.items():
        if isinstance(df_missing, pd.DataFrame):    # Confirm df_missing is a DataFrame before data manipulation
            #Set up our empty list (filtered_rows) to store the rows with missing data in our selected columns
            selected_data = df_missing[filtered_columns_to_show].copy() # Create new DF with our selected columns
            selected_data.insert(0, "DataFrame Name", df_semester)      # Insert new column for the Dataframe Name
            filtered_rows.append(selected_data)                         # Add to list filtered_rows by appending selected_data

    # Print results
    if filtered_rows:
        missing_data_summary = pd.concat(filtered_rows, ignore_index=True) # Transform our list (filtered_rows) to DF
    
        print("\nIn-Person Classes without Dates or Start/End Times:")
        print(missing_data_summary)

else:
    print("No missing schedule info found!")"

    """

'\nfor df_semester, df in dfs.items():\n    condition = (                                   # Conditions: In Person & not Independent Study           \n        (df["Instruction Mode"] == "In Person") &  \n        (df["Component"] != "Independent Study")) \n    \n    # Find missing observations ("-") in our days_times_columns columns\n    missing_days_times = df[condition & df[days_times_columns].isin(["-"]).any(axis=1)] # .any() for any column with "-", not just 3 for 3\n    \n    # If missing data exists ("-"), store in missing_data_report\n    if not missing_days_times.empty:\n        missing_data_report[df_semester] = missing_days_times.copy()\n\n# Print results\nif missing_data_report:\n    for df_semester, df_missing in missing_data_report.items():\n        if isinstance(df_missing, pd.DataFrame):    # Confirm df_missing is a DataFrame before data manipulation\n            #Set up our empty list (filtered_rows) to store the rows with missing data in our selected columns\n         

?? When I run it again, it finds 10 new courses it didn't find before that are missing a day/time

Print Each DF's unique values by column & their respective counts

In [317]:
for df_semester, df in dfs.items():
    print(f"\n=== DataFrame: {df_semester} ===")
    for col in df.columns:
        print(f"\nColumn: {col}")
        value_counts = df[col].value_counts(dropna=False)
        print(value_counts)



=== DataFrame: df_fall21 ===

Column: Term
Term
Fall 2021    283
Name: count, dtype: int64

Column: Session
Session
Regular Academic Session    214
Seven Week - First           39
Seven Week - Second          30
Name: count, dtype: int64

Column: Session Code
Session Code
1      214
7W1     39
7W2     30
Name: count, dtype: int64

Column: Campus
Campus
University of Arizona - Main    189
Arizona Online                   94
Name: count, dtype: int64

Column: Facility
Facility
Online                        211
Live Online                    15
Elec & Comp Engr, Rm 107       12
R P Harvill Bldg, Rm 460        7
Haury Anthro Bldg, Rm 216       4
Modern Languages, Rm 311        4
R P Harvill Bldg, Rm 130        4
R P Harvill Bldg, Rm 402        3
R P Harvill Bldg, Rm 415        2
R P Harvill Bldg, Rm 401        2
R P Harvill Bldg, Rm 319        2
Aero & Mech Engr, Rm S324       2
R P Harvill Bldg, Rm 232        2
R P Harvill Bldg, Rm 134        2
R P Harvill Bldg, Rm 204        1
R P Harvi

Course Description
Intro to Digital Cultures         10
Digital Storytelling & Culture    10
Statistic Foundations Info Age     9
Digital Dilemmas                   8
Computational Thinking & Doing     8
                                  ..
Special Topics                     1
Graduate Seminar                   1
Installation Design                1
Natural Language Processing        1
Esports Casting                    1
Name: count, Length: 89, dtype: int64

Column: Class Status Code
Class Status Code
A    298
Name: count, dtype: int64

Column: Start Date
Start Date
2023-01-11    248
2023-03-13     50
Name: count, dtype: int64

Column: End Date
End Date
2023-05-03    251
2023-03-03     47
Name: count, dtype: int64

Column: Class Meeting Number
Class Meeting Number
1    298
Name: count, dtype: Int64

Column: Meeting Days
Meeting Days
-     242
TR     32
MW     10
M       6
T       3
W       3
F       2
Name: count, dtype: int64

Column: Meeting Time Start
Meeting Time Start
00:00:00  

Explore Co-Convened Classes

In [318]:
for df_semester, df in dfs.items():
    # Find for co-convened courses
    co_convened_rows = df[df["Combined Section"] != "-"] # Any value besides "-" is co-convened

    # Count total number of co-convened courses (rows) & how many are unique ("combined section")
    co_convened_total = co_convened_rows.shape[0]
    co_convened_unique = co_convened_rows["Combined Section"].nunique()

    print(f"{df_semester}: {co_convened_total} co-convened courses, {co_convened_unique} unique co-convened course groups (Combined Sections)")


df_fall21: 223 co-convened courses, 95 unique co-convened course groups (Combined Sections)
df_fall22: 222 co-convened courses, 88 unique co-convened course groups (Combined Sections)
df_fall23: 251 co-convened courses, 103 unique co-convened course groups (Combined Sections)
df_fall24: 261 co-convened courses, 104 unique co-convened course groups (Combined Sections)
df_spring22: 219 co-convened courses, 92 unique co-convened course groups (Combined Sections)
df_spring23: 244 co-convened courses, 92 unique co-convened course groups (Combined Sections)
df_spring24: 267 co-convened courses, 108 unique co-convened course groups (Combined Sections)
df_spring25: 281 co-convened courses, 112 unique co-convened course groups (Combined Sections)


Now that data is cleaned, Combine into Fall, Spring, and All DataFrames

In [319]:
#Combined DataFrames: Fall, Spring, and All
df_fall_all = pd.concat([df_fall21, df_fall22, df_fall23, df_fall24],ignore_index=True)
df_spring_all = pd.concat([df_spring22, df_spring23, df_spring24, df_spring25], ignore_index=True)
df_all = pd.concat([df_fall_all, df_spring_all], ignore_index=True)

dfs_cleaned = {
    "df_fall_all": df_fall_all,
    "df_spring_all": df_spring_all,
    "df_all": df_all
    }

In [320]:
#Save DataFrames to Excel (.xlsx)

for df_semester, df in dfs_cleaned.items():
    filename = f"{df_semester}.xlsx"        # Save file name
    df.to_excel(filename, index=False)      # Save to Excel
    print(f"Saved {df_semester} to {filename}")


Saved df_fall_all to df_fall_all.xlsx
Saved df_spring_all to df_spring_all.xlsx
Saved df_all to df_all.xlsx


Print Unique Values by Columns again, now for our combined dataframes

In [322]:
for df_semester, df in dfs_cleaned.items():
    print(f"\n=== DataFrame: {df_semester} ===")
    for col in df.columns:
        print(f"\nColumn: {col}")
        value_counts = df[col].value_counts(dropna=False)
        print(value_counts)


=== DataFrame: df_fall_all ===

Column: Term
Term
Fall 2024    433
Fall 2023    415
Fall 2021    346
Fall 2022    345
Name: count, dtype: int64

Column: Session
Session
Regular Academic Session    1175
Seven Week - First           187
Seven Week - Second          177
Name: count, dtype: int64

Column: Session Code
Session Code
1      1175
7W1     187
7W2     177
Name: count, dtype: int64

Column: Campus
Campus
University of Arizona - Main    1054
Arizona Online                   485
Name: count, dtype: int64

Column: Facility
Facility
Online                      978
Not Available               220
Room To Be Announced         53
Live Online                  31
R P Harvill Bldg, Rm 402     23
                           ... 
Modern Languages, Rm 413      1
M Pacheco ILC, Rm 125         1
R P Harvill Bldg, Rm 102      1
R P Harvill Bldg, Rm 105      1
C E Chavez Bldg, Rm 109       1
Name: count, Length: 92, dtype: int64

Column: Room Capacity
Room Capacity
1      1013
0       220
99     

Make more DataFrames: 1) Undergrad vs Graduate, 2) Online vs In-Person